# IMPORT LIBARY

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import tkinter as tk

In [ ]:
data_path = 'datasets/supermarket_sales.csv'

df = pd.read_csv(data_path)
 
 
# melihat 5 data teratas
df.head(5)

In [ ]:
# melihat tipe data pada tiap kolom header
df.info()

In [ ]:
# mendeskirpsikan isi dari datasets
df.describe()